## Lesson 5: Human in the Loop

Lesson objective: Get feedback on answers from a human operator

In this lab, you'll learn how to make Workflows easy to iterate on answers to the questionnaire by getting feedback on answers from the human operator and re-answering when necessary.

In [1]:
import json
import os

from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.base.base_query_engine import BaseQueryEngine
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows
from llama_parse import LlamaParse, ResultType

In [2]:
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
llama_cloud_api_key = os.environ["LLAMA_CLOUD_API_KEY"]

In [3]:
import nest_asyncio

nest_asyncio.apply()

### Adding a feedback loop

The changes you're going to make here are:
1. Use the `InputRequiredEvent` and `HumanResponseEvent`, new special events specifically designed to allow you to exit the workflow, and get feedback back into it.
2. You used to have a single step which parsed your form and fired off all your questions. Since we now might loop back and ask questions several times, we don't need to parse the form every time, so we'll split up those steps. This kind of refactoring is very common as you create a more complex workflow:
   - Your new `generate_questions` step will be triggered either by a `GenerateQuestionsEvent`, triggered by the form parser, or by a `FeedbackEvent`, which is the loop we'll take after getting feedback.
3. `fill_in_application` will emit an `InputRequiredEvent`, and in the `external_step` you'll wait for a `HumanResponseEvent`. This will pause the whole workflow waiting for outside input.
4. Finally, you'll use the LLM to parse the feedback and decide whether it means you should continue and output the results, or if you need to loop back.

In [4]:
# new!
from llama_index.core.workflow import InputRequiredEvent, HumanResponseEvent

In [5]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str
    field: str
    
class ResponseEvent(Event):
    field: str
    response: str

# new!
class FeedbackEvent(Event):
    feedback: str

class GenerateQuestionsEvent(Event):
    pass

In [ ]:
class RAGWorkflow(Workflow):

    storage_dir: str = "./storage"
    llm: OpenAI
    query_engine: BaseQueryEngine

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = OpenAI(
            model="gpt-4.1-nano",
            api_key=openai_api_key,
            api_base=os.environ["OPENAI_API_BASE"],
            temperature=0.5,
        )

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(
                storage_context=storage_context,
                embed_model=OpenAIEmbedding(
                    model_name="Cohere-embed-v3-english",
                    api_key=openai_api_key,
                    api_base=os.environ["OPENAI_API_BASE"],
                ),
            )
        else:
            # parse and load the resume document
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                # base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type=ResultType.MD,
                user_prompt="This is a resume, gather related facts together and format it as bullet points with headers",
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents=documents,
                embed_model=OpenAIEmbedding(
                    model_name="Cohere-embed-v3-english",
                    api_key=openai_api_key,
                    api_base=os.environ["OPENAI_API_BASE"],
                ),
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # you no longer need a query to be passed in,
        # you'll be generating the queries instead
        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    # new - separated the form parsing from the question generation
    @step
    async def parse_form(
        self, ctx: Context, ev: ParseFormEvent
    ) -> GenerateQuestionsEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            # base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type=ResultType.MD,
            user_prompt="This is a job application form."
            " Create a list of all the fields that need to be filled in."
            " Return a bulleted list of the fields ONLY.",
        )

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            "This is a parsed form. Convert it into a JSON object containing only the list of fields to be filled in,"
            f" in the form {{ fields: [...] }}. <form>{result.text}</form>. Return JSON ONLY, no markdown."
        )
        fields = json.loads(raw_json.text)["fields"]

        await ctx.store.set("fields_to_fill", fields)

        return GenerateQuestionsEvent()

    # new - this step can get triggered either by GenerateQuestionsEvent or a FeedbackEvent
    @step
    async def generate_questions(
        self, ctx: Context, ev: GenerateQuestionsEvent | FeedbackEvent
    ) -> QueryEvent:

        # get the list of fields to fill in
        fields = await ctx.store.get("fields_to_fill")

        # generate one query for each of the fields, and fire them off
        for field in fields:
            question = f"How would you answer this question about the candidate? <field>{field}</field>"
            ctx.send_event(QueryEvent(field=field, query=question))

        # store the number of fields so we know how many to wait for later
        await ctx.store.set("total_fields", len(fields))

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(
            f"This is a question about the specific resume we have in our database: {ev.query}"
        )
        return ResponseEvent(field=ev.field, response=response.response)

    # new - we now emit an InputRequiredEvent
    @step
    async def fill_in_application(
        self, ctx: Context, ev: ResponseEvent
    ) -> InputRequiredEvent | None:
        # get the total number of fields to wait for
        total_fields = await ctx.store.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return  # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join(
            "Field: " + r.field + "\n" + "Response: " + r.response for r in responses
        )

        result = self.llm.complete(
            f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """
        )

        # new! save the result for later
        await ctx.store.set("filled_form", str(result))

        # new! Let's get a human in the loop
        return InputRequiredEvent(
            prefix="How does this look? Give me any feedback you have on any of the answers.",
            result=result,
        )

    # new! Accept the feedback.
    @step
    async def get_feedback(
        self, ctx: Context, ev: HumanResponseEvent
    ) -> FeedbackEvent | StopEvent:

        result = self.llm.complete(
            f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
        """
        )

        verdict = result.text.strip()

        print(f"LLM says the verdict was {verdict}")
        if verdict == "OKAY":
            return StopEvent(result=await ctx.store.get("filled_form"))

        return FeedbackEvent(feedback=ev.response)

Okay! Your workflow is now ready to get some feedback, but how do we actually get it? The `InputRequiredEvent` is an event in the event stream, just like the `ProgressEvents` and `TextEvents` you've seen in lesson 2. You can intercept it the same way you did those, and use the `send_event` method on the context to send back a `HumanResponseEvent`.

In [7]:
workflow = RAGWorkflow(timeout=600, verbose=False)

handler = workflow.run(
    resume_file="./data/fake_resume.pdf",
    application_form="./data/fake_application_form.pdf",
)

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        print("We've filled in your form! Here are the results:\n")
        print(event.result)

        # now ask for input from the keyboard
        response = input(event.prefix)
        handler.ctx.send_event(HumanResponseEvent(response=response))

response = await handler

print("Agent complete! Here's your final result:")
print(str(response))

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/index_store.json.
Started parsing the file under job_id 20b52d0a-70de-4749-924b-411ae17a7cd2
We've filled in your form! Here are the results:

- **Position:** Full Stack Developer with experience as Senior Full Stack Developer, specializing in building scalable web applications, microservices, and leading technical teams.  
- **First Name:** Sarah  
- **Last Name:** Chen  
- **Email:** sarah.chen@email.com  
- **Phone:** Not provided  
- **LinkedIn:** linkedin.com/in/sarahchen  
- **Project Portfolio:** Developed a full-stack carbon footprint tracking app with machine learning features recognized in a top environmental app list; created a real-time encrypted chat app serving over 5,000 users; contributed to open source projects; built multiple responsive web applications using various frameworks and technologies.  
- **Degree:** Ba

### Using the Feedback

Okay! Now let's further modify things to actually do something useful with the feedback in `generate_questions` step. This involves checking if there's feedback, and appending it to the questions. In this simple example, we're going to append the feedback to every question in case it's relevant, but a more sophisticated agent might apply it only to the fields where the feedback applied.

In [8]:
class RAGWorkflow(Workflow):

    storage_dir: str = "./storage"
    llm: OpenAI
    query_engine: BaseQueryEngine

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = OpenAI(
            model="gpt-4.1-nano",
            api_key=openai_api_key,
            api_base=os.environ["OPENAI_API_BASE"],
            temperature=0.5,
        )

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(
                storage_context=storage_context,
                embed_model=OpenAIEmbedding(
                    model_name="Cohere-embed-v3-english",
                    api_key=openai_api_key,
                    api_base=os.environ["OPENAI_API_BASE"],
                ),
            )
        else:
            # parse and load the resume document
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                # base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type=ResultType.MD,
                user_prompt="This is a resume, gather related facts together and format it as bullet points with headers",
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents=documents,
                embed_model=OpenAIEmbedding(
                    model_name="Cohere-embed-v3-english",
                    api_key=openai_api_key,
                    api_base=os.environ["OPENAI_API_BASE"],
                ),
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    # form parsing
    @step
    async def parse_form(
        self, ctx: Context, ev: ParseFormEvent
    ) -> GenerateQuestionsEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            # base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type=ResultType.MD,
            user_prompt="This is a job application form."
            " Create a list of all the fields that need to be filled in."
            " Return a bulleted list of the fields ONLY.",
        )

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            "This is a parsed form. Convert it into a JSON object containing only the list of fields to be filled in,"
            f" in the form {{ fields: [...] }}. <form>{result.text}</form>. Return JSON ONLY, no markdown."
        )
        fields = json.loads(raw_json.text)["fields"]

        await ctx.store.set("fields_to_fill", fields)

        return GenerateQuestionsEvent()

    # generate questions
    @step
    async def generate_questions(
        self, ctx: Context, ev: GenerateQuestionsEvent | FeedbackEvent
    ) -> QueryEvent:

        # get the list of fields to fill in
        fields = await ctx.store.get("fields_to_fill")

        # generate one query for each of the fields, and fire them off
        for field in fields:
            question = f"How would you answer this question about the candidate? <field>{field}</field>"

            # new! Is there feedback? If so, add it to the query:
            if hasattr(ev, "feedback"):
                question += f"""
                    \nWe previously got feedback about how we answered the questions.
                    It might not be relevant to this particular field, but here it is:
                    <feedback>{ev.feedback}</feedback>
                """

            ctx.send_event(QueryEvent(field=field, query=question))

        # store the number of fields so we know how many to wait for later
        await ctx.store.set("total_fields", len(fields))

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(
            f"This is a question about the specific resume we have in our database: {ev.query}"
        )
        return ResponseEvent(field=ev.field, response=response.response)

    # Get feedback from the human
    @step
    async def fill_in_application(
        self, ctx: Context, ev: ResponseEvent
    ) -> InputRequiredEvent | None:
        # get the total number of fields to wait for
        total_fields = await ctx.store.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return  # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join(
            "Field: " + r.field + "\n" + "Response: " + r.response for r in responses
        )

        result = self.llm.complete(
            f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """
        )

        # save the result for later
        await ctx.store.set("filled_form", str(result))

        # Fire off the feedback request
        return InputRequiredEvent(
            prefix="How does this look? Give me any feedback you have on any of the answers.",
            result=result,
        )

    # Accept the feedback when a HumanResponseEvent fires
    @step
    async def get_feedback(
        self, ctx: Context, ev: HumanResponseEvent
    ) -> FeedbackEvent | StopEvent:

        result = self.llm.complete(
            f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
        """
        )

        verdict = result.text.strip()
        print(f"LLM says the verdict was {verdict}")

        if verdict == "OKAY":
            return StopEvent(result=await ctx.store.get("filled_form"))

        return FeedbackEvent(feedback=ev.response)

In [9]:
workflow = RAGWorkflow(timeout=600, verbose=False)
handler = workflow.run(
    resume_file="./data/fake_resume.pdf",
    application_form="./data/fake_application_form.pdf",
)

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        print("We've filled in your form! Here are the results:\n")
        print(event.result)

        # now ask for input from the keyboard
        response = input(event.prefix)
        handler.ctx.send_event(HumanResponseEvent(response=response))

response = await handler

print("Agent complete! Here's your final result:")
print(str(response))

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/index_store.json.
Started parsing the file under job_id 5267a51c-4462-45bf-8d77-a47ed24de7ca
We've filled in your form! Here are the results:

- **Position:** The candidate has experience as a Full Stack Developer, including roles such as Senior Full Stack Developer and Full Stack Developer at various companies.  
- **First Name:** Sarah  
- **Last Name:** Chen  
- **Email:** sarah.chen@email.com  
- **Phone:** Not provided in the available information  
- **LinkedIn:** linkedin.com/in/sarahchen  
- **Project Portfolio:** Includes EcoTrack, a full-stack app for tracking carbon footprints with machine learning features, featured in TechCrunch's "Top 10 Environmental Impact Apps of 2023"; and ChatFlow, a real-time chat app using WebSocket with end-to-end encryption, serving over 5,000 monthly active users.  
- **Degree:** Bachelor of

### Workflow Visualization

In [10]:
WORKFLOW_FILE = "workflows/feedback_workflow.html"
draw_all_possible_flows(workflow, filename=WORKFLOW_FILE)

workflows/feedback_workflow.html
